# Bibliotecas usadas

In [2]:
import numpy as np
import pandas as pd
from itertools import combinations

# Leer datos


In [3]:

datos = np.load('spotify.npy', encoding = "latin1",allow_pickle="False") # Cargar archivo


## Ordenamos las listas de reproduccion dentro de un arreglo

In [4]:
playlists=[]
for i in datos.item():
    datos.item()[i].sort()
    playlists.append(list((set(datos.item()[i]))))

### Mostramos los datos

In [5]:
playlists

[['Toxic',
  'Livin on Sunday',
  "Jumpin', Jumpin'",
  'Soak Up The Sun',
  'Replay',
  'The Great Escape',
  'Party In The U.S.A.',
  'Everytime We Touch - Radio Edit',
  'Your Love Is My Drug',
  'Somebody To Love',
  'Baby',
  'Beep',
  'Check On It - feat. Bun B and Slim Thug',
  'Forever',
  'Ice Box',
  'Me & U',
  'My Boo',
  'Crazy In Love',
  'Right Where You Want Me - Radio Edit Version',
  'Mr. Brightside',
  'Paper Planes',
  'Year 3000',
  "Leavin'",
  'Sk8er Boi',
  'Where Is The Love?',
  'Dirty Little Secret',
  'My Happy Ending',
  'Rock Your Body',
  'All The Small Things',
  "It Wasn't Me",
  'This Is Me',
  'Just The Girl',
  "Stacy's Mom",
  'Promiscuous',
  'Say My Name',
  'Yo (Excuse Me Miss)',
  'A Thousand Miles',
  'Buttons',
  'How Do You Sleep? - Featuring Ludacris',
  'Run It!',
  'Check Yes Juliet',
  'Lip Gloss',
  'Miss Independent',
  'One Less Lonely Girl',
  'Lose Control (feat. Ciara & Fat Man Scoop)',
  'Beautiful Soul',
  'Whatcha Say',
  'Yeah!'

# Algoritmo Apriori

## Funcion para crear el primer nivel, recorriendo todas las canciones

In [7]:
def primerNivel(datos,min):
    listaCancionesTodas = {} 
    listaAparicion={}
    newMin=min*len(datos)
    for playlist in datos:
      for grupoCanciones in playlist:
        listaCancionesTodas[grupoCanciones] = listaCancionesTodas.get(grupoCanciones, 0) + 1
        listaAparicion[grupoCanciones]=listaAparicion.get(grupoCanciones, [])+[datos.index(playlist)]
    listaCancionesPasan=[]
    listaAparicionPasan={}
    for dato in listaAparicion:
      if(len(listaAparicion[dato])>=newMin):
        listaCancionesPasan.append([dato])
        listaAparicionPasan[tuple([dato])]=listaAparicion.get(dato)
    lista1=sorted(listaAparicionPasan.items())
    return lista1


## Funcion usada para la creacion de un nuevo nivel a partir del anterior

In [8]:
def crearNivel(datosPrevios,tamañoDatos,min,nivel):
  nuevoNivel={}
  numDatosPrevios=len(datosPrevios)
  newMin=min*tamañoDatos
  listaAparicion={}
  for i in range(numDatosPrevios):
    for j in range(i+1,numDatosPrevios):
      L1 = datosPrevios[i][0] [: nivel]
      L2 = datosPrevios[j][0] [: nivel] 
      L11=sorted(L1)
      L22=sorted(L2)
      if L11==L22:
        nuevoElemento=set(datosPrevios [i][0]) | set(datosPrevios [j][0])
        subconjunto=list(combinations(list(nuevoElemento),nivel+1))
        conjuntoAparicion=set(datosPrevios[i][1])&set(datosPrevios[j][1])
        nuevoNivel[tuple(nuevoElemento)]=list(conjuntoAparicion)
  listaPasan={}
  for key in nuevoNivel:
    if(len(nuevoNivel[key])>=newMin):
      listaPasan[key]=nuevoNivel.get(key)
  
  lista1=sorted(listaPasan.items())
  return lista1
    

## Creacion de los Itemsets con Apriori

In [9]:
def apriori(datos,suport):
    inicio = primerNivel(datos,suport)
    tamañoDatos=len(datos)
    ListaItemsets = [inicio]
    k = 0
    indicador=True
    while (indicador):
        nivelK = crearNivel(ListaItemsets[k],tamañoDatos, suport,k)
        if(len(nivelK)>0):
            ListaItemsets.append(nivelK)
        else:
            indicador=False
        k += 1

    #for i in range(len(ListaItemsets))
    return ListaItemsets

### Encontramos los itemsets frecuentes

In [10]:
itemsets=apriori(playlists,0.015)

### Mostramos los itemsets encontrados y cuantos niveles tiene


In [11]:
for i in range( len(itemsets)):
    print("- Elementos del nivel",i+1,"son ",len(itemsets[i]))

- Elementos del nivel 1 son  349
- Elementos del nivel 2 son  22


### Mostramos el soporte de los elementos

In [12]:
for i in range(len(itemsets)):
    print("========== Itemsets de ",i+1,"elementos ==========")
    for elemento in itemsets[i]:
        print("-",elemento[0],"tienen un soporte de: ",len(elemento[1])/len(playlists))

========== Itemsets de  1 elementos ==========
- ("'Till I Collapse",) tienen un soporte de:  0.0169
- ('0 To 100 / The Catch Up',) tienen un soporte de:  0.0161
- ('1-800-273-8255',) tienen un soporte de:  0.0264
- ('24K Magic',) tienen un soporte de:  0.0167
- ('679 (feat. Remy Boyz)',) tienen un soporte de:  0.0244
- ('7 Years',) tienen un soporte de:  0.0188
- ('7/11',) tienen un soporte de:  0.015
- ('Africa',) tienen un soporte de:  0.0168
- ('Again',) tienen un soporte de:  0.0212
- ("Ain't No Mountain High Enough",) tienen un soporte de:  0.0222
- ('Alive',) tienen un soporte de:  0.0224
- ('All I Want',) tienen un soporte de:  0.0221
- ('All My Friends',) tienen un soporte de:  0.0186
- ('All Night',) tienen un soporte de:  0.0153
- ('All Night (feat. Knox Fortune)',) tienen un soporte de:  0.0166
- ('All Star',) tienen un soporte de:  0.0166
- ('All The Small Things',) tienen un soporte de:  0.0165
- ('All Time Low',) tienen un soporte de:  0.0189
- ('All of Me',) tienen un s

## Reglas de asociacion

In [13]:
def getReglasdeAsociacion(itemsetsFrecuentes,minConfianza,minLift):
  Reglas=[]
  confianzas=[]
  lifts=[]
  #empezamos a recorrer los itemsets de manera inversa para tener los itemsets de mayor tamaño
  # diccionario de suport
  suportDict={}
  for nivel in itemsetsFrecuentes:
    for elemento in nivel:
      suportDict[elemento[0]]=len(elemento[1])/10000
  tamañoMax=len(itemsets)-1
  
  while(tamañoMax>=1):
    for itemset in itemsets[tamañoMax]:
      combinaciones=list(combinations(itemset[0],tamañoMax))
      for combinacion in combinaciones:
        confianza=suportDict[itemset[0]]/suportDict[combinacion]
        diferencia=tuple(set(itemset[0]).difference(set(combinacion)))
        lift=suportDict[itemset[0]]/(suportDict[combinacion]*suportDict[diferencia])
        #print(lift)
        if(confianza>=minConfianza and lift>minLift):
          Reglas.append([combinacion,diferencia])
          confianzas.append(confianza)
          lifts.append(lift)
    tamañoMax-=1
  listaFinal=[]
  for i in range(len(Reglas)):
    aux=[Reglas[i],confianzas[i],lifts[i]]
    listaFinal.append(aux)
  return listaFinal



## Calculamos las reglas de asociacion

In [23]:
reglas=getReglasdeAsociacion(itemsets,0.51,11)

### Mostramos las reglas junto a su confianza y lift

In [30]:
for i in range(len(reglas)):
    print("====== Regla ",i+1," ======")
    print("regla: ",reglas[i][0][0],"->",reglas[i][0][1])
    print("confianza: ",reglas[i][1])
    print("lift: ",reglas[i][2])

====== Regla  1  ======
regla:  ('Bounce Back',) -> ('Bad and Boujee (feat. Lil Uzi Vert)',)
confianza:  0.569023569023569
lift:  16.493436783291852
====== Regla  2  ======
regla:  ('Bounce Back',) -> ('HUMBLE.',)
confianza:  0.5252525252525252
lift:  11.295753231237102
====== Regla  3  ======
regla:  ('Congratulations',) -> ('HUMBLE.',)
confianza:  0.531017369727047
lift:  11.41972838122682
====== Regla  4  ======
regla:  ('DNA.',) -> ('HUMBLE.',)
confianza:  0.8225108225108225
lift:  17.68840478517898
====== Regla  5  ======
regla:  ('Mask Off',) -> ('HUMBLE.',)
confianza:  0.6455696202531646
lift:  13.883217639853001
====== Regla  6  ======
regla:  ('XO TOUR Llif3',) -> ('HUMBLE.',)
confianza:  0.631578947368421
lift:  13.582342954159593
====== Regla  7  ======
regla:  ('goosebumps',) -> ('HUMBLE.',)
confianza:  0.5422077922077921
lift:  11.660382628124562
====== Regla  8  ======
regla:  ('Mask Off',) -> ('Congratulations',)
confianza:  0.5126582278481012
lift:  12.721047837421864
=

# Conclusiones e interpretaciones

las reglas encontradas cumplen con la condicion de que todas las asociaciones son artistas de rap y 
el que tiene mayor confianza indica de que el cantantees el compositor de las dos canciones